# Static elastic constants
This Notebook follows the workflow for setting up, running, and processing the elastic_constants_static calculations.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os
from datetime import date
from math import floor
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://bokeh.pydata.org/
import bokeh
import bokeh.plotting
import bokeh.resources
import bokeh.io
import bokeh.models

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.2


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('demo')

## 1. Run elastic_constants_static calculations

This calculation evaluates the elastic constants for a crystal using small strain deformations.

In [3]:
calculation = iprPy.load_calculation('elastic_constants_static')
run_directory = iprPy.load_run_directory('demo_1')

### Show calculation's allowed keys

These are the keys that the input script will accept.

In [4]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'strainrange', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


### Write input script

In [5]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information from relax_dynamic results
buildcombos                 atomicparent load_file parent
parent_record               calculation_crystal_space_group
parent_load_key             unit-cell-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   5 5 5

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
strainrange                  1e-5
strainrange                  1e-6
strainrange                  1e-7
strainrange                  1e-8
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [6]:
# Parse input script
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_name list using filtered results

In [7]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

144


In [8]:
print(list(unique_crystals.keys()))

['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id', 'composition', 'prototype', 'family', 'method', 'transformed', 'E_coh', 'a', 'b', 'c', 'alpha', 'beta', 'gamma']


In [9]:
unique_crystals = unique_crystals[unique_crystals.potential_LAMMPS_id == '2012--Park-H--Mo--LAMMPS--ipr1']
#unique_crystals = unique_crystals[unique_crystals.composition == '']
#unique_crystals = unique_crystals[unique_crystals.prototype == '']
print(len(unique_crystals))

35


In [10]:
input_dict['parent_name'] = unique_crystals.calc_key.tolist()

### Prepare calculations

In [11]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 140 of style calculation_elastic_constants_static
 - 0 are complete
 - 140 still to run
 - 0 issued errors


### Run calculations

In [ ]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 13384


In [17]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
un.py", line 110, in run
    output = sp.check_output(mpi_command + lammps_command + extra + [\'-in\'] + script_name)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command \'[\'lmp_mpi\', \'-in\', \'cij.in\']\' returned non-zero exit status 1.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_elastic_constants_static.py", line 282, in <module>
    main(*sys.argv[1:])
  File "calc_elastic_constants_static.py", line 55, in main
    dmax = input_dict[\'maxatommotion\'])
  File "calc_elastic_constants_static.py", line 161, in elastic_constants_static
    output = lmp.run(lammps_command, lammps_script, mpi_command)
un.py", line 116, in run
 

## 2. Calculation analysis

### Retrieve finished calculation results

In [4]:
records = database.get_records_df(style='calculation_elastic_constants_static', full=True, flat=False, status='finished')
pot_records = database.get_records_df(style='potential_LAMMPS')


### Build raw data

In [28]:

for record in records.itertuples():
    
    

### Save raw data

In [24]:
records

,C,LAMMPS_version,error,family,iprPy_version,key,load_file,load_options,load_style,potential_LAMMPS_id,...,potential_key,raw_cij_negative,raw_cij_positive,script,sizemults,status,strainrange,symbols,composition,prototype
0,[[ 1.00178860e+00 5.19970006e-01 5.19970001e...,26 Jan 2017-ICMS,NaN,oqmd-1214769,0.8.2,04ff09b8-045e-449f-96a8-5e2f4e293b81,e28fe811-3c5f-48f8-afc9-bf4c5eab15ca.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[0.9981299782567745, 0.519784983869613, 0.519...","[[1.0054472117620832, 0.5201550242286447, 0.52...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-08,[Ag],Ag,oqmd-1214769
1,[[7.75440013e-01 5.85913566e-01 5.85913566e-01...,26 Jan 2017-ICMS,NaN,oqmd-676271,0.8.2,058e5d76-ee0e-4d15-825b-bde1d57d8ff4,c1ec7d94-51af-442e-a228-1344a6b4bd4d.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[0.7754400402332803, 0.5859135960201266, 0.58...","[[0.7754399854977791, 0.5859135359955076, 0.58...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-08,[Ag],Ag,A1--Cu--fcc
2,[[9.15509391e-01 5.87837102e-01 4.68786910e-01...,26 Jan 2017-ICMS,NaN,mp-10597,0.8.2,067076cb-b446-46d3-85d6-57a5a2801170,8b349f7d-3991-438a-82d9-a54ea4aff2a2.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[0.9155093880846337, 0.587837159089215, 0.468...","[[0.9155093946073352, 0.5878370369853919, 0.46...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-08,[Ag],Ag,A3--Mg--hcp
3,[[ 1.71884125 0.11664781 0.11664781 0. ...,26 Jan 2017-ICMS,NaN,Ah--alpha-Po--sc,0.8.2,0965b376-851a-4937-8d22-e008b73693b6,235949ee-c95e-42d4-811e-97cd38ae26ba.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[1.7188557949937802, 0.1166481994723816, 0.11...","[[1.7188266955196796, 0.11664742379996358, 0.1...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-06,[Ag],Ag,Ah--alpha-Po--sc
5,[[1.23002056e+00 3.34497866e-01 3.34497867e-01...,26 Jan 2017-ICMS,NaN,A15--beta-W,0.8.2,0b5546cd-40a3-4a16-9e7b-36eb4b320725,2e05623b-4aa8-4fa4-9dde-23ea692a264e.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[1.2300206002192275, 0.3344978810216979, 0.33...","[[1.2300205134954774, 0.3344978477689239, 0.33...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-08,[Ag],Ag,A15--beta-W
6,[[-1.16578046e+01 2.04301977e+02 7.60693662e...,26 Jan 2017-ICMS,NaN,oqmd-1215927,0.8.2,14320f07-07d2-4658-9c5c-ebbb60e80f28,b795f6b7-e112-4955-8aef-707826274b06.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[-771.562260178292, -531.5306901329793, -1878...","[[748.246650987803, 1148.5868161952337, 1908.3...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-05,[Ag],Ag,oqmd-1215927
7,[[-8.43526260e-01 2.67322314e+00 2.47887670e...,26 Jan 2017-ICMS,NaN,oqmd-1215571,0.8.2,17a2d439-c6e1-4ad1-baea-603c029741c7,51683d18-53ca-4458-a255-a89662a2b950.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[-0.8437595810493815, 2.6753583961808216, 0.2...","[[-0.8432929380640785, 2.675064538151388, 0.24...",calc_elastic_constants_static,"[[0, 3], [0, 3], [0, 3]]",finished,1.000000e-05,[Ag],Ag,A5--beta-Sn
8,[[0.9155094 0.58783711 0.46878691 0. ...,26 Jan 2017-ICMS,NaN,mp-10597,0.8.2,1b6d8d82-2b48-4f63-8739-212c25bb1e8d,8b349f7d-3991-438a-82d9-a54ea4aff2a2.json,key unit-cell-atomic-system,system_model,2006--Williams-P-L--Ag--LAMMPS--ipr1,...,ce32faaf-5d57-4861-a2ec-efbfc63f07e6,"[[0.9155097856544541, 0.5878372246520955, 0.46...","[[0.91

In [51]:
data = []
for record in records.itertuples():
    crystal = unique_crystals[unique_crystals.calc_key == record.load_file.split('.')[0]].iloc[0]
    
    dat = {}
    dat['calc_key'] = record.key
    dat['potential_LAMMPS_key'] = crystal.potential_LAMMPS_key
    dat['potential_LAMMPS_id'] = crystal.potential_LAMMPS_id
    dat['potential_key'] = crystal.potential_key
    dat['potential_id'] = crystal.potential_id
    dat['prototype'] = crystal.prototype
    dat['family'] = crystal.family
    dat['composition'] = crystal.composition
    dat['strainrange'] = record.strainrange
    dat['a'] = crystal.a
    dat['b'] = crystal.b
    dat['c'] = crystal.c
    dat['alpha'] = crystal.alpha
    dat['beta'] = crystal.beta
    dat['gamma'] = crystal.gamma

    dat['straindirection'] = 'positive'
    cij = uc.get_in_units(record.raw_cij_positive, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

    dat['straindirection'] = 'negative'
    cij = uc.get_in_units(record.raw_cij_negative, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

columns = ['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id',
           'composition', 'prototype', 'family', 
           'a', 'b', 'c', 'alpha', 'beta', 'gamma',
           'strainrange', 'straindirection', 
           'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
           'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
           'C31', 'C32', 'C33', 'C34', 'C35', 'C36',
           'C41', 'C42', 'C43', 'C44', 'C45', 'C46',
           'C51', 'C52', 'C53', 'C54', 'C55', 'C56',
           'C61', 'C62', 'C63', 'C64', 'C65', 'C66']

data = pd.DataFrame(data, columns=columns).sort_values(['potential_LAMMPS_id', 'composition', 'prototype', 'strainrange', 'straindirection'])

In [52]:
data.to_csv('elastic-constants.csv')

In [46]:
for i in range(6):
    for j in range(6):
        print(i, j, record.raw_cij_positive[i,j] - record.raw_cij_positive[j,i])

0 0 0.0
0 1 0.014007833025681315
0 2 1.213081024520335e-05
0 3 -9.267338712355426e-06
0 4 3.664908123963368e-05
0 5 -1.0116550891086139e-05
1 0 -0.014007833025681315
1 1 0.0
1 2 -0.013995781938079488
1 3 -5.917426740206227e-06
1 4 -1.3989912938078078e-06
1 5 -7.328756466577149e-06
2 0 -1.213081024520335e-05
2 1 0.013995781938079488
2 2 0.0
2 3 -9.347732214957515e-06
2 4 4.426728055781595e-05
2 5 -1.3428046564262807e-05
3 0 9.267338712355426e-06
3 1 5.917426740206227e-06
3 2 9.347732214957515e-06
3 3 0.0
3 4 1.3617138553361802e-11
3 5 7.857531994214562e-11
4 0 -3.664908123963368e-05
4 1 1.3989912938078078e-06
4 2 -4.426728055781595e-05
4 3 -1.3617138553361802e-11
4 4 0.0
4 5 -1.78485620492013e-12
5 0 1.0116550891086139e-05
5 1 7.328756466577149e-06
5 2 1.3428046564262807e-05
5 3 -7.857531994214562e-11
5 4 1.78485620492013e-12
5 5 0.0



### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [22]:
# Add calculation data to PotentialProperties records
for implememtation_key in np.unique(records.potential_LAMMPS_key):
    imp_records = records[records.potential_LAMMPS_key == implememtation_key].sort_values('family')
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation_id = pot_record.id
    potential_id = pot_record.pot_id
    potential_key = pot_record.pot_key
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['elastic-constants'] = DM()
    for composition in np.unique(imp_records.composition):
        content['per-potential-properties']['cohesive-energy-scan'].append('composition', composition)
    
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())